### Unsupervised Classification Analysis
### Team Transparency, CSE 6242 Spring 2020
#### Marc Boulet

This notebook will take the merged dataset of country features and attempt to rank them through unsupervised methods (k-means clustering, hierarchical clustering, k-nearest neighbours, DBSCAN, PCA of grouped attributes, etc).

In [ ]:
import pandas as pd
import difflib
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)